In [18]:
import cv2
from PIL import Image, ImageEnhance

In [29]:
boundingBox = []
centroid = []
#aspectRatio = []

#increase brightness
im = Image.open("data/fractions1.jpg")
#rotated_im = im.rotate(-90)
enhancer = ImageEnhance.Brightness(im)
enhanced_im = enhancer.enhance(1.5)
enhanced_im.save("data/bright.png")

# read the image
image = cv2.imread("data/bright.png")

image[image >= 127] = 255
image[image < 127] = 0

#rotate if required
#image = cv2.rotate(image, rotateCode = 0)

origWidth, origHeight, channels = image.shape

# apply greyscale filter onto image
image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# apply a blur to remove noise from the image
image_grey = cv2.GaussianBlur(image_grey, (7, 7), 0)

# threshold image to inverted binary - black and white only
ret, thresh1 = cv2.threshold(image_grey ,127,255,cv2.THRESH_BINARY_INV)
# dilate image to make white portions thicker
image_dilate = cv2.dilate(thresh1, None, iterations=2)

cv2.imwrite("separated/PROCESSED.png",image_dilate)

# find ALL contours in the image
contours = cv2.findContours(image_dilate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

original = image.copy()
i = 0

for contour in contours:
    # if contour area is too small, ignore it
    if(cv2.contourArea(contour) < 1000): continue
    
    # bounding rectangles placed on all contours detected
    x,y,w,h = cv2.boundingRect(contour)
    #if x is 0: continue
    if w*h < 25: continue
        
    # region of interest taken from contour - we are interested in the contents of the rectangle only
    roi = image[y:y+h, x:x+w]

    boundingBox.insert(i, [x,y,w,h])
    #print ("ORIGINAL")
    #print (boundingBox[i])
    
    centroidX = (x+w)/2
    centroidY = (y+h)/2
    
    #centroidX = x + (w/2)
    #centroidY = y - (h/2)
    centroid.insert(i, [centroidX,centroidY])
    #print (centroid[i])
    
    # rectangles marked onto image#DEBUG
    #cv2.rectangle(original, (x,y), (x+w,y+h), (0,255,0), 2)
    
    # save the contents of each rectangle as a separate PNG image
    cv2.imwrite("separated/roi"+ str(i) + ".png", roi)

    i = i + 1

#bubble sort
for passNum in range(0,i):
    for c in range(0, i-passNum-1):
        if(boundingBox[c][0]>boundingBox[c+1][0]):
            temp = boundingBox[c]
            boundingBox[c] = boundingBox[c+1]
            boundingBox[c+1] = temp
            temp2 = centroid[c]
            centroid[c] = centroid[c+1]
            centroid[c+1] = temp2
            
for n in range(0,i):
    print ("Sorted BoundingBox",n)
    print (boundingBox[n])
    print (centroid[n])
    #print (aspectRatio[n])
    #print (centroid[n])
    
    #find key features
    if(n<=i and n>1):
        aspectRatio = (boundingBox[n][2]/boundingBox[n][3])
        #fractions
        if(aspectRatio>=5.00):
            #denominator
            x = boundingBox[n][0]
            y = boundingBox[n][1]
            w = boundingBox[n][2]
            h = boundingBox[n][3]
            boxH = h
            y = y + h
            h = origHeight - y
            h = origHeight - h
            roi = image[y:y+h, x:x+w]
            cv2.imwrite("separated/denom"+ str(i) + ".png", roi)
            
            #numerator
            h = origHeight-h
            h = h-boxH
            y = y - boxH - h

            roi = image[y:y+h, x:x+w]
            cv2.imwrite("separated/num"+ str(i) + ".png", roi)

            #rest
            x = x + w
            w = origWidth-x
            h = origHeight

            roi = image[y:y+h, x:x+w]
            cv2.imwrite("separated/rest"+ str(i) + ".png", roi)
        
        #square roots
        if(boundingBox[n-1][0]<=(boundingBox[n][0]+boundingBox[n][2])<=(boundingBox[n-1][0]+boundingBox[n-1][2])):
            if(boundingBox[n-1][1]<=(boundingBox[n][1]+boundingBox[n][3])<=(boundingBox[n-1][1]+boundingBox[n-1][3])):
                print("square root found")
        else:
            #exponents
            if(centroid[n][0]>centroid[n-1][0]):
                if(centroid[n][1]<centroid[n-1][1]-40):
                        print("exponent found")                                                                            
                            
#cv2.imshow("IMG", original)      
cv2.waitKey(0)

Sorted BoundingBox 0
[842, 1830, 1727, 67]
[1284.5, 948.5]
Sorted BoundingBox 1
[999, 1483, 318, 322]
[658.5, 902.5]
Sorted BoundingBox 2
[1412, 1467, 195, 272]
[803.5, 869.5]
Sorted BoundingBox 3
[1488, 2000, 177, 379]
[832.5, 1189.5]
Sorted BoundingBox 4
[1712, 1467, 216, 233]
[964.0, 850.0]
exponent found


-1